In [ ]:
# This is a semantic search approach
# It's basically a visualization tool (not really) for ocular inspection
# It takes a fixed number of pages but should be updated to a sliding window of maybe three pages at a time
# Input is, for now, (1) one issue of one single newspaper as well as (2) the gold standard, which is minimal, and (3) a query paragraph (text box)
# Output is n recommended text boxes, printed side by side to be able to compare them

In [6]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
import numpy as np

In [8]:
df = pd.read_csv("data/gold_standard.csv")

In [9]:
model = SentenceTransformer("KBLab/sentence-bert-swedish-cased")

In [10]:
# TODO
# Implement a sliding window of pages

In [11]:
corpus = df["content_x"].tolist()[0:18] # use only three pages for now; improves performance

In [12]:
embeddings = model.encode(corpus, convert_to_tensor=True)

In [13]:
# query text
query = ["Nyheter. DN kan i dag avslöja nya = uppgifter kring SD-riksdagsleda= moten Jonas Millard, vars Flash: back-konto spridit grovt hat mot : kvinnor. Lösenordet till kontot är 1 samma som Millard använt till flera = andra nättjänster - däribland sin I jobbmejl - ett lösenord Millard på5 står sig ha delat med personer som han inte vet vilka de är. Reaktionerna från andra riks- "]

In [14]:
def print_side_by_side(a, b, size=40, space=1):
    while a or b:
        print(a[:size].ljust(size) + "\t" + b[:size])
        a = a[size:]
        b = b[size:]
        # TODO
        # make output more readable

In [15]:
def guess(query, n):
    # Find the closest n sentences of the corpus for each query sentence based on cosine similarity
    top_k = min(n, len(embeddings))
    guess = []

    query_embedding = model.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, embeddings)[0] 
    top_results = torch.topk(cos_scores, k=top_k) # we use cosine-similarity and torch.topk to find the highest n scores
    
    for score, idx in zip(top_results[0], top_results[1]):
        s = corpus[idx] + " (SCORE: {:.4f})".format(score)
        guess.append(s)
    return guess

In [16]:
def answer(query, df):
    df.article_id = df.article_id.astype(int)
    query_row = df[df['content_x'].str.contains(query[0])]
    query_article_id = query_row["article_id"][0]
    article_match_df = df.loc[df['article_id'] == query_article_id]
    answer = article_match_df["content_x"].tolist()
    return answer

In [17]:
n = 3
g = guess(query, n)
a = answer(query, df[0:18])
print("QUERY:", query[0])
print("\nCORRECT ANSWER\t\t\t\t\tMOST SIMILAR PARAGRAPHS")

guess_string = "\n\n".join(g)
answer_string = "\n\n".join(a)

print_side_by_side(answer_string, guess_string)

QUERY: Nyheter. DN kan i dag avslöja nya = uppgifter kring SD-riksdagsleda= moten Jonas Millard, vars Flash: back-konto spridit grovt hat mot : kvinnor. Lösenordet till kontot är 1 samma som Millard använt till flera = andra nättjänster - däribland sin I jobbmejl - ett lösenord Millard på5 står sig ha delat med personer som han inte vet vilka de är. Reaktionerna från andra riks- 

CORRECT ANSWER					MOST SIMILAR PARAGRAPHS
Nyheter. DN kan i dag avslöja nya = uppg	Nyheter. DN kan i dag avslöja nya = uppg
ifter kring SD-riksdagsleda= moten Jonas	ifter kring SD-riksdagsleda= moten Jonas
 Millard, vars Flash: back-konto spridit	 Millard, vars Flash: back-konto spridit
 grovt hat mot : kvinnor. Lösenordet til	 grovt hat mot : kvinnor. Lösenordet til
l kontot är 1 samma som Millard använt t	l kontot är 1 samma som Millard använt t
ill flera = andra nättjänster - däriblan	ill flera = andra nättjänster - däriblan
d sin I jobbmejl - ett lösenord Millard 	d sin I jobbmejl - ett lösenord Millard 

/tmp/ipykernel_54269/1994656438.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.article_id = df.article_id.astype(int)
